In [1]:
# Janei Elliston

!pip3 install dmba 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 67.0 MB/s eta 0:00:00


In [2]:
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pylab as plt
from dmba import regressionSummary, classificationSummary
from dmba import liftChart, gainsChart

no display found. Using non-interactive Agg backend


In [4]:
data = pd.read_csv('BostonHousing-1.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV,CAT. MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0,0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6,0
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7,1
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4,1
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2,1


In [8]:
# Fit a multiple linear regression model to the median house price (MEDV) as a function of CRIM, CHAS, and RM.
X = data[['CRIM', 'CHAS', 'RM']]
y = data['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
model = LinearRegression()
model.fit(X_train, y_train)

coef = model.coef_
intercept = model.intercept_
eq = f'MEDV = {coef[0]:.4f} * CRIM + {coef[1]:.4f} * CHAS + {coef[2]:.4f} * RM + {intercept:.4f}'
print(eq)


MEDV = -0.2545 * CRIM + 2.8258 * CHAS + 7.9445 * RM + -26.5628


In [10]:
regressionSummary(y, model.predict(X))


Regression statistics

                      Mean Error (ME) : -0.1083
       Root Mean Squared Error (RMSE) : 6.1569
            Mean Absolute Error (MAE) : 4.1997
          Mean Percentage Error (MPE) : -8.4022
Mean Absolute Percentage Error (MAPE) : 23.0757


What is the equation for predicting the median house price from the predictors in the model?

MEDV = -0.2545 * CRIM + 2.8258 * CHAS + 7.9445 * RM + -26.5628


In [12]:
#Using the estimated regression model, what median house price is predicted for a tract in the Boston area that does not bound the Charles River, has a crime rate of 0.1, and where the average number of rooms per house is 6?
MHP = -0.2545 * 0.1 + 2.8258 * 0 + 7.9445 * 6 + -26.5628
print(MHP)


21.078750000000003


The predicted median house price (MEDV) is $21,079

In [13]:
#  Compute the correlation table for the 12 numerical predictors and search for highly correlated pairs. These have potential redundancy and can cause multi-collinearity. Choose which ones to remove based on this table.
correlation_matrix = data.iloc[:, :-1].corr()
print(correlation_matrix)

             CRIM        ZN     INDUS      CHAS       NOX        RM       AGE  \
CRIM     1.000000 -0.200469  0.406583 -0.055892  0.420972 -0.219247  0.352734   
ZN      -0.200469  1.000000 -0.533828 -0.042697 -0.516604  0.311991 -0.569537   
INDUS    0.406583 -0.533828  1.000000  0.062938  0.763651 -0.391676  0.644779   
CHAS    -0.055892 -0.042697  0.062938  1.000000  0.091203  0.091251  0.086518   
NOX      0.420972 -0.516604  0.763651  0.091203  1.000000 -0.302188  0.731470   
RM      -0.219247  0.311991 -0.391676  0.091251 -0.302188  1.000000 -0.240265   
AGE      0.352734 -0.569537  0.644779  0.086518  0.731470 -0.240265  1.000000   
DIS     -0.379670  0.664408 -0.708027 -0.099176 -0.769230  0.205246 -0.747881   
RAD      0.625505 -0.311948  0.595129 -0.007368  0.611441 -0.209847  0.456022   
TAX      0.582764 -0.314563  0.720760 -0.035587  0.668023 -0.292048  0.506456   
PTRATIO  0.289946 -0.391679  0.383248 -0.121515  0.188933 -0.355501  0.261515   
LSTAT    0.455621 -0.412995 

Based on this correlation matrix, we can see that TAX and RAD are highly correlated (> 0.91), so we can remove one of these variables.

In [18]:
data = data.drop('RAD', axis=1)

In [34]:
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score


In [43]:
X = data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'LSTAT']]
y = data['MEDV']
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

def train_model(variables):
  model = LinearRegression()
  model.fit(train_X[variables], train_y)
  return model
def score_model(model, variables):
  return AIC_score(train_y, model.predict(train_X[variables]), model) 
allVariables = train_X.columns
best_model1, best_variables = backward_elimination(allVariables, train_model,score_model, verbose=True)
print(best_variables)
regressionSummary(valid_y, best_model1.predict(valid_X[best_variables]))



Variables: CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, TAX, PTRATIO, LSTAT
Start: score=1815.90
Step: score=1813.90, remove AGE
Step: score=1811.94, remove TAX
Step: score=1810.16, remove INDUS
Step: score=1810.16, remove None
['CRIM', 'ZN', 'CHAS', 'NOX', 'RM', 'DIS', 'PTRATIO', 'LSTAT']

Regression statistics

                      Mean Error (ME) : -0.0397
       Root Mean Squared Error (RMSE) : 5.1713
            Mean Absolute Error (MAE) : 3.6519
          Mean Percentage Error (MPE) : -5.2055
Mean Absolute Percentage Error (MAPE) : 17.1988


In [44]:
def train_model(variables):
  if len(variables) == 0:
    return None
  model = LinearRegression()
  model.fit(train_X[variables], train_y)
  return model
def score_model(model, variables):
  if len(variables) == 0:
    return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
  return AIC_score(train_y, model.predict(train_X[variables]), model)
best_model2, best_variables = forward_selection(train_X.columns, train_model, score_model, verbose=True)
print(best_variables)

Variables: CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, TAX, PTRATIO, LSTAT
Start: score=2191.75, constant
Step: score=1934.91, add LSTAT
Step: score=1874.18, add RM
Step: score=1842.54, add PTRATIO
Step: score=1837.69, add CHAS
Step: score=1835.00, add NOX
Step: score=1817.90, add DIS
Step: score=1811.82, add ZN
Step: score=1810.16, add CRIM
Step: score=1810.16, add None
['LSTAT', 'RM', 'PTRATIO', 'CHAS', 'NOX', 'DIS', 'ZN', 'CRIM']


In [45]:
best_model3, best_variables = stepwise_selection(train_X.columns, train_model, score_model,
verbose=True)
print(best_variables)


Variables: CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, TAX, PTRATIO, LSTAT
Start: score=2191.75, constant
Step: score=1934.91, add LSTAT
Step: score=1874.18, add RM
Step: score=1842.54, add PTRATIO
Step: score=1837.69, add CHAS
Step: score=1835.00, add NOX
Step: score=1817.90, add DIS
Step: score=1811.82, add ZN
Step: score=1810.16, add CRIM
Step: score=1810.16, unchanged None
['LSTAT', 'RM', 'PTRATIO', 'CHAS', 'NOX', 'DIS', 'ZN', 'CRIM']


In [46]:
plt.hist(train_y - best_model1.predict(train_X), bins='auto', label='Backward Elimination')
plt.hist(train_y - best_model2.predict(train_X), bins='auto', label='Forward Selection')
plt.hist(train_y - best_model3.predict(train_X), bins='auto', label='Stepwise Selection')
plt.legend()
plt.xlabel('Error')
plt.ylabel('Frequency')
plt.show()

ValueError: ignored